In [24]:
#creating spark session

In [25]:
from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
    .master("local[*]")
    .appName("Spark Streaming")
    .getOrCreate()
)

In [26]:
spark

In [30]:
#read input data

df_raw = spark.read.format("text").load("data/example.txt")

In [31]:
df_raw.printSchema()

root
 |-- value: string (nullable = true)



In [32]:
df_raw.show(10,False)

+----------------------------------------+
|value                                   |
+----------------------------------------+
|spark structured streaming is intersting|
+----------------------------------------+



In [33]:
#splite the line into words
from pyspark.sql.functions import split

df_words = df_raw.withColumn("words", split("value",' '))

In [34]:
df_words.show()

+--------------------+--------------------+
|               value|               words|
+--------------------+--------------------+
|spark structured ...|[spark, structure...|
+--------------------+--------------------+



In [35]:
#explod the list of words
from pyspark.sql.functions import explode

df_explode = df_words.withColumn("word",explode("words")).drop("value","words")

In [36]:
df_explode.show()

+----------+
|      word|
+----------+
|     spark|
|structured|
| streaming|
|        is|
|intersting|
+----------+



In [37]:
#aggregate the words to generate count
from pyspark.sql.functions import count,lit

df_agg = df_explode.groupBy("word").agg(count(lit(1)).alias("cnt"))

In [38]:
df_agg.show()

+----------+---+
|      word|cnt|
+----------+---+
| streaming|  1|
|        is|  1|
|     spark|  1|
|structured|  1|
|intersting|  1|
+----------+---+

